In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
import matplotlib.pyplot as plt
from pandasql import sqldf

In [3]:
df_cars = pd.read_csv("./vw.csv")
df_cars.head(3)

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
0,T-Roc,2019,25000,Automatic,13904,Diesel,145,49.6,2.0
1,T-Roc,2019,26883,Automatic,4562,Diesel,145,49.6,2.0
2,T-Roc,2019,20000,Manual,7414,Diesel,145,50.4,2.0


In [6]:
# Definimos la función para los "queries"
pysqldf = lambda q: sqldf(q, globals())

In [8]:
query = """SELECT model, transmission, fuelType FROM df_cars"""
dfCars = pysqldf(query)
dfCars.head(3)

,model,transmission,fuelType
0,T-Roc,Automatic,Diesel
1,T-Roc,Automatic,Diesel
2,T-Roc,Manual,Diesel


In [9]:
dfCars["fuelTypeEncd"] = dfCars.fuelType.astype("category").cat.codes

In [10]:
dfCars.sample(3)

,model,transmission,fuelType,fuelTypeEncd
3672,Golf,Semi-Auto,Petrol,3
427,T-Roc,Manual,Petrol,3
204,T-Roc,Manual,Petrol,3
